In [2]:
spark

res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2fa90773


In [3]:
spark.version

res1: String = 2.3.1


# Kaggle Dataset Load

In [4]:
val input = sc.textFile("Scorecard.csv").map(line =>line.split(","))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at <console>:25


### Create a header index of the dataset

In [5]:
val header = sc.parallelize(input.take(1))

header: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[3] at parallelize at <console>:27


In [6]:
val headerIndex = header.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

headerIndex: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[6] at zipWithIndex at <console>:29


In [7]:
//headerIndex.coalesce(1).saveAsTextFile("index")

Originally, I exported and imported the data. Not sure why.

In [8]:
// header.map(_.mkString(",")).coalesce(1).saveAsTextFile("header")

In [9]:
// val headerLoad = sc.textFile("header.csv")

In [10]:
// val headerList= headerLoad.flatMap(line => line.split(","))

In [11]:
// val headerIndex = headerList.zipWithIndex

In [12]:
val idxSat = headerIndex.lookup("SAT_AVG")(0).toInt

idxSat: Int = 60


# DoE Dataset Load Long way

In [13]:
/* This isn't working for some reason below when I try to grab the UST RDD

val in2016 = sc.textFile("MERGED2016_17_PP.csv")

val marked2016 = in2016.map(_.mkString(",")).map(x => x + ",2016").map(line =>line.split(","))

val in2015 = sc.textFile("MERGED2015_16_PP.csv")

val marked2015 = in2015.map(_.mkString(",")).map(x => x + ",2015").map(line =>line.split(","))

val in2014 = sc.textFile("MERGED2014_15_PP.csv")

val marked2014 = in2014.map(_.mkString(",")).map(x => x + ",2014").map(line =>line.split(","))

val in2013 = sc.textFile("MERGED2013_14_PP.csv")

val marked2013 = in2016.map(_.mkString(",")).map(x => x + ",2013").map(line =>line.split(","))

val in2012 = sc.textFile("MERGED2012_13_PP.csv")

val marked2012 = in2012.map(_.mkString(",")).map(x => x + ",2012").map(line =>line.split(","))

val in2011 = sc.textFile("MERGED2011_12_PP.csv")

val marked2011 = in2011.map(_.mkString(",")).map(x => x + ",2011").map(line =>line.split(","))

val in2010 = sc.textFile("MERGED2010_11_PP.csv")

val marked2010 = in2010.map(_.mkString(",")).map(x => x + ",2010").map(line =>line.split(","))

val in2009 = sc.textFile("MERGED2009_10_PP.csv")

val marked2009 = in2009.map(_.mkString(",")).map(x => x + ",2009").map(line =>line.split(","))

val in2008 = sc.textFile("MERGED2008_09_PP.csv")

val marked2008 = in2008.map(_.mkString(",")).map(x => x + ",2008").map(line =>line.split(","))

val in2007 = sc.textFile("MERGED2007_08_PP.csv")

val marked2007 = in2007.map(_.mkString(",")).map(x => x + ",2007").map(line =>line.split(","))

*/

# DoE Dataset Load concise

In [14]:

val in2016 = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",2016").map(line =>line.split(","))

//val marked2016 = in2016.map(_.mkString(","))

val in2015 = sc.textFile("MERGED2015_16_PP.csv").map(x => x + ",2015").map(line =>line.split(","))

//val marked2015 = in2015.map(_.mkString(","))

val in2014 = sc.textFile("MERGED2014_15_PP.csv").map(x => x + ",2014").map(line =>line.split(","))

//val marked2014 = in2014.map(_.mkString(","))

val in2013 = sc.textFile("MERGED2013_14_PP.csv").map(x => x + ",2013").map(line =>line.split(","))

//val marked2013 = in2016.map(_.mkString(","))

val in2012 = sc.textFile("MERGED2012_13_PP.csv").map(x => x + ",2012").map(line =>line.split(","))

//val marked2012 = in2012.map(_.mkString(","))

val in2011 = sc.textFile("MERGED2011_12_PP.csv").map(x => x + ",2011").map(line =>line.split(","))

//val marked2011 = in2011.map(_.mkString(","))

val in2010 = sc.textFile("MERGED2010_11_PP.csv").map(x => x + ",2010").map(line =>line.split(","))

//val marked2010 = in2010.map(_.mkString(","))

val in2009 = sc.textFile("MERGED2009_10_PP.csv").map(x => x + ",2009").map(line =>line.split(","))

//val marked2009 = in2009.map(_.mkString(","))

val in2008 = sc.textFile("MERGED2008_09_PP.csv").map(x => x + ",2008").map(line =>line.split(","))

//val marked2008 = in2008.map(_.mkString(","))

val in2007 = sc.textFile("MERGED2007_08_PP.csv").map(x => x + ",2007").map(line =>line.split(","))

//val marked2007 = in2007.map(_.mkString(","))


in2016: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12] at map at <console>:25
in2015: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[16] at map at <console>:29
in2014: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[20] at map at <console>:33
in2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[24] at map at <console>:37
in2012: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[28] at map at <console>:41
in2011: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[32] at map at <console>:45
in2010: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[36] at map at <console>:49
in2009: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[40] at map at <console>:53
in2008: org.apache.spark.rdd.RDD[Array[String...

In [15]:
// Test the year concat function. 
// marked2016.coalesce(1).saveAsTextFile("2016test")
in2008.take(2)

res8: Array[Array[String]] = Array(Array(UNITID, OPEID, OPEID6, INSTNM, CITY, STABBR, ZIP, ACCREDAGENCY, INSTURL, NPCURL, SCH_DEG, HCM2, MAIN, NUMBRANCH, PREDDEG, HIGHDEG, CONTROL, ST_FIPS, REGION, LOCALE, LOCALE2, LATITUDE, LONGITUDE, CCBASIC, CCUGPROF, CCSIZSET, HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, NANTI, MENONLY, WOMENONLY, RELAFFIL, ADM_RATE, ADM_RATE_ALL, SATVR25, SATVR75, SATMT25, SATMT75, SATWR25, SATWR75, SATVRMID, SATMTMID, SATWRMID, ACTCM25, ACTCM75, ACTEN25, ACTEN75, ACTMT25, ACTMT75, ACTWR25, ACTWR75, ACTCMMID, ACTENMID, ACTMTMID, ACTWRMID, SAT_AVG, SAT_AVG_ALL, PCIP01, PCIP03, PCIP04, PCIP05, PCIP09, PCIP10, PCIP11, PCIP12, PCIP13, PCIP14, PCIP15, PCIP16, PCIP19, PCIP22, PCIP23, PCIP24, PCIP25, PCIP26, PCIP27, PCIP29, PCIP30, PCIP31, PCIP38, PCIP39, PCIP40, PCIP41, PCIP4...

## Compile the 10 year RDD.

In [16]:
val scoreCard10Year = sc.union(in2016, in2015, in2014, in2013, in2012, in2011, in2010, in2009, in2008, in2007)
val twoYear = in2016.union(in2015)
scoreCard10Year.take(1)
twoYear.take(1)

scoreCard10Year: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[49] at union at <console>:36
twoYear: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[50] at union at <console>:37
res9: Array[Array[String]] = Array(Array(UNITID, OPEID, OPEID6, INSTNM, CITY, STABBR, ZIP, ACCREDAGENCY, INSTURL, NPCURL, SCH_DEG, HCM2, MAIN, NUMBRANCH, PREDDEG, HIGHDEG, CONTROL, ST_FIPS, REGION, LOCALE, LOCALE2, LATITUDE, LONGITUDE, CCBASIC, CCUGPROF, CCSIZSET, HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, NANTI, MENONLY, WOMENONLY, RELAFFIL, ADM_RATE, ADM_RATE_ALL, SATVR25, SATVR75, SATMT25, SATMT75, SATWR25, SATWR75, SATVRMID, SATMTMID, SATWRMID, ACTCM25, ACTCM75, ACTEN25, ACTEN75, ACTMT25, ACTMT75, ACTWR25, ACTWR75, ACTCMMID, ACTENMID, ACTMTMID, ACTWRMID, SAT_AVG, SAT_AVG_ALL, PCIP01, PCIP03, PCIP04, PCI...

### Create a header index of the dataset

In [17]:
val header2016 = sc.parallelize(in2016.take(1))

val header2016Index = header2016.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2016Index.coalesce(1).saveAsTextFile("index2016") //Textfiles only made once

val header2007 = sc.parallelize(in2007.take(1))

val header2007Index = header2007.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2007Index.coalesce(1).saveAsTextFile("index2007") // Textfiles only made once

header2016: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[51] at parallelize at <console>:28
header2016Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[54] at zipWithIndex at <console>:30
header2007: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[55] at parallelize at <console>:34
header2007Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[58] at zipWithIndex at <console>:36


## Count the rows of the kaggle compiled dataset

In [24]:
input.count()

res14: Long = 124700


In [25]:
input.map( x => x(1)).distinct().count()

res15: Long = 10708


In [28]:
// val year = input.map( x => x(1730))

// year.take(1100).foreach(println)

// It looks like there are a lot of blank lines in there early on in the dataset. 

val notBlankYear = input.filter(x => x(1730) != "")

notBlankYear.count()

year: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[72] at map at <console>:29
notBlankYear: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[73] at filter at <console>:35
res18: Long = 24630


In [ ]:
notBlankYear.map(x => x(1730)).take(20)

notBlankYear.filter(x => x(1730) contains "2013").count()

## Count of rows for the compiled DoE Scorecard

In [19]:
scoreCard10Year.count()

res11: Long = 74181


# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years

In [20]:
val ust = scoreCard10Year.filter(x => x(0) contains "174914")

ust: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[59] at filter at <console>:38


In [21]:
ust.take(1)

res12: Array[Array[String]] = Array(Array(174914, 00234500, 002345, University of St Thomas, Saint Paul, MN, 55105-1078, Higher Learning Commission, www.stthomas.edu, www.stthomas.edu/financialaid/undergraduate/resources/netpricecalculator/, NULL, 0, 1, 1, 3, 4, 2, 27, 4, 11, NULL, 44.943439, -93.188986, 17, 10, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0.8266, 0.82655521620318, 500, 660, 550, 630, NULL, NULL, 580, 590, NULL, 24, 29, 23, 29, 24, 28, NULL, NULL, 27, 26, 26, NULL, 1204, 1204, 0, 0.0037, 0, 0.0007, 0.0504, 0, 0.0212, 0, 0.0358, 0.0651, 0, 0.0073, 0.0066, 0, 0.0139, 0.0066, 0, 0.0658, 0.0117, 0, 0.0102, 0, 0.0417, 0, 0.0197, 0, 0.0336, 0, 0.0117, 0.0621, 0, 0, 0, 0, 0.0058, 0.0417, 0.4751, 0.0095, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...

In [22]:
ust.count()

res13: Long = 10


In [31]:
val ustHeader = header2016.union(ust)
ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

ustHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[81] at union at <console>:46


# Create an index of the header 

Really had a tough time figuring out how to get the header loaded as just a string. Loading it from the original source results in type RDD[Array[String]], so I ended up using a CSV file that I outputed from the original input. Ended up just needing a **flatMap** to keep it in the correct format. 

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [ ]:
val mnColleges = input.filter( x => x(5) contains "MN").filter( x => x(1730) contains "2013")

# Regional Schools  
This RDD contains schools from the upper-midwest (Minnesota, Wisconsin, North Dakota, South Dakota, Iowa, Nebraska). This RDD will be used for regional comparisons. 

In [ ]:
val upMidWestColleges = scoreCard10Year.filter( x => x(19) contains "Plains")

# TextFile Saves

In [ ]:
headerIndex.foreach {
    case(field, ind) => println(s"$field is $ind")
}

In [ ]:
ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

In [ ]:
headerIndex.map(_.mkString(",")).coalesce(1).saveAsTextFile("headerIndex")

In [ ]:
mnColleges.map(_.mkString(",")).coalesce(1).saveAsTextFile("Minnesota")

In [ ]:
val regionalHeader = header.union(upMidWestColleges)

In [ ]:
regionalHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("regionalHeader")